In [1]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)

Windows-10-10.0.19045-SP0
Python 3.10.5 (tags/v3.10.5:f377153, Jun  6 2022, 16:14:13) [MSC v.1929 64 bit (AMD64)]


In [2]:
class cfg:
    seed = 42
    nfolds = 5
    njobs = 2

In [3]:
import numpy as np
import pandas as pd
import os

import lazypredict
from lazypredict.Supervised import LazyRegressor

from sklearn.model_selection import train_test_split

In [4]:
# load data
train = pd.read_csv('../data/final/train.csv')
valid = pd.read_csv('../data/final/valid.csv')
test = pd.read_csv('../data/final/test.csv')

train.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal,IsSynthetic,...,Monterey County,Napa County,Orange County,Other,Riverside County,San Francisco County,Santa Barbara County,Santa Clara County,Ventura County,Yolo County
0,5.09,17.00,6.19,1.11,1538.00,3.37,38.02,-121.36,1.03,1.00,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
1,3.36,5.00,4.24,1.02,3278.00,2.07,33.68,-117.53,1.97,1.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,5.04,29.00,6.60,1.02,1119.00,2.71,33.84,-117.95,2.64,1.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,4.14,22.00,5.45,0.99,1750.00,3.02,38.47,-121.77,1.19,1.00,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
4,1.60,39.00,4.74,1.08,1168.00,3.25,36.75,-119.80,0.67,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [5]:
# quick info
TARGET = 'MedHouseVal'
FEATURES = [c for c in train.columns if c not in [TARGET]]

print(f'Target: {TARGET}\nFeatures: {FEATURES}')
print('Train set shape:', train.shape)
print('Valid set shape:', valid.shape)
print('Test set shape:', test.shape)

Target: MedHouseVal
Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude', 'IsSynthetic', 'AnomalyScore', 'rot_15_x', 'rot_15_y', 'rot_30_x', 'rot_30_y', 'rot_45_x', 'pca_lon', 'pca_lat', 'Alameda County', 'Contra Costa County', 'Fresno County', 'Kern County', 'Los Angeles County', 'Monterey County', 'Napa County', 'Orange County', 'Other', 'Riverside County', 'San Francisco County', 'Santa Barbara County', 'Santa Clara County', 'Ventura County', 'Yolo County']
Train set shape: (42922, 33)
Valid set shape: (14855, 33)
Test set shape: (24759, 32)


In [6]:
# sample train set
smp = train.sample(10000, random_state=cfg.seed)

# get features and target
x, y = smp[FEATURES], smp[TARGET]
x_val, y_val = valid[FEATURES], valid[TARGET]

In [7]:
# select regressors
list(enumerate(n[0] for n in lazypredict.Supervised.REGRESSORS))
idx = [0, 1, 3, 4, 6, 7, 8, 12, 13, 14, 19, 23, 24 , 25, 26, 33, 35, 36, 37, 40, 41]
mask = [True if est in idx else False for est in range(len(lazypredict.Supervised.REGRESSORS))]

from itertools import compress
lazypredict.Supervised.REGRESSORS = list(compress(lazypredict.Supervised.REGRESSORS, mask))
lazypredict.Supervised.REGRESSORS

[('AdaBoostRegressor', sklearn.ensemble._weight_boosting.AdaBoostRegressor),
 ('BaggingRegressor', sklearn.ensemble._bagging.BaggingRegressor),
 ('DecisionTreeRegressor', sklearn.tree._classes.DecisionTreeRegressor),
 ('DummyRegressor', sklearn.dummy.DummyRegressor),
 ('ElasticNetCV', sklearn.linear_model._coordinate_descent.ElasticNetCV),
 ('ExtraTreeRegressor', sklearn.tree._classes.ExtraTreeRegressor),
 ('ExtraTreesRegressor', sklearn.ensemble._forest.ExtraTreesRegressor),
 ('HistGradientBoostingRegressor',
  sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingRegressor),
 ('HuberRegressor', sklearn.linear_model._huber.HuberRegressor),
 ('KNeighborsRegressor', sklearn.neighbors._regression.KNeighborsRegressor),
 ('LassoCV', sklearn.linear_model._coordinate_descent.LassoCV),
 ('LinearRegression', sklearn.linear_model._base.LinearRegression),
 ('LinearSVR', sklearn.svm._classes.LinearSVR),
 ('MLPRegressor', sklearn.neural_network._multilayer_perceptron.MLPRe

In [8]:
# lazypredict
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(x, x_val, y, y_val)

100%|██████████| 21/21 [03:42<00:00, 10.57s/it]


In [9]:
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
LGBMRegressor,0.75,0.75,0.57,0.57
HistGradientBoostingRegressor,0.75,0.75,0.57,2.64
ExtraTreesRegressor,0.75,0.75,0.58,18.02
RandomForestRegressor,0.74,0.74,0.59,67.81
XGBRegressor,0.73,0.73,0.60,2.05
BaggingRegressor,0.71,0.71,0.62,6.90
MLPRegressor,0.71,0.71,0.62,21.16
NuSVR,0.69,0.69,0.65,49.71
SVR,0.69,0.69,0.65,37.58
